In [1]:
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import folium
import sys
import concurrent.futures
from tqdm import tqdm
from threading import Semaphore
import json
import folium
import openrouteservice
import json
import time
import openrouteservice
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import shape
from geopy.distance import geodesic
from shapely import wkt
from openrouteservice.exceptions import ApiError
sys.path.append('../scripts')
from utilities import get_vic_subs_as_csv

In [2]:
client = ors.Client(key='5b3ce3597851110001cf624828f2b63314b24af798ed8608d8464d19')

In [16]:
input_file = '../data/raw/current_rent_info.json'
#Modify this outfut path to genrate 3 json files.  '../data/raw/coordinates_by_address_first_4500.json', and '../data/raw/coordinates_by_address_second_4500.json'
output_file = '../data/raw/coordinates_by_address_final_4500.json'
with open(input_file, 'r') as file:
    data = json.load(file)

In [17]:
# Define the rate limit
RATE_LIMIT = 100  # Requests per minute
REQUEST_INTERVAL = 60 / RATE_LIMIT  # Interval between requests in seconds

# Semaphore to control the number of concurrent requests
semaphore = Semaphore(3)  # Max number of concurrent threads
def get_coordinates(address):
    try:
        with semaphore:
            response = client.pelias_search(address)
            if response['features']:
                coords = response['features'][0]['geometry']['coordinates']
                return {'latitude': coords[1], 'longitude': coords[0]}
            else:
                return {'latitude': None, 'longitude': None}
    except Exception as e:
        print(f"Error fetching coordinates for address '{address}': {e}")
        return {'latitude': None, 'longitude': None}

def fetch_coordinates_for_item(url_details):
    url, details = url_details
    address = details.get('name')
    if address:
        coords = get_coordinates(address)
        details['coordinates'] = coords

def worker(url_details):
    fetch_coordinates_for_item(url_details)
    # Delay to prevent exceeding rate limits
    time.sleep(REQUEST_INTERVAL)

In [18]:
with open(input_file, 'r') as file:
    data = json.load(file)
    
data_items = list(data.items())[9000: ]


In [ ]:
# Find coordinates based on address for houses
max_threads = 3

with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    # Create a list of futures
    futures = [executor.submit(worker, (url, details)) for url, details in data_items]

    # Wrap futures with tqdm to show progress
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing items"):
        try:
            future.result()
        except Exception as e:
            print(f"An error occurred: {e}")

# Save the updated data to a new JSON file
with open(output_file, 'w') as file:
    json.dump(data, file, indent=4)

print(f"Updated data with coordinates saved to {output_file}")

In [ ]:
with open('../data/raw/properties_with_coords.json', 'r') as file:
    data = json.load(file)
coordinates = [
    [details['coordinates']['longitude'], details['coordinates']['latitude']]
    for url, details in list(data.items()) if details.get('coordinates')
]
print(coordinates)

In [4]:
# #Find closest train station
# for coords in coordinates[0:1]:
#     pois = client.places(
#         request='pois',  
#         geojson={"type": "Point", "coordinates": coords},
#         filter_category_ids=[604], 
#         buffer=2000,
#         limit=1, 
#         sortby='distance'
#     )
    
#     print(f"POIs for coordinates {coords}:")
    
#     if 'features' in pois:
#         top_pois = pois['features'][:1]
#         print(top_pois)
#     else:
#         print("No POIs found.")

POIs for coordinates [144.983539, -37.812236]:
[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [144.983838, -37.816492]}, 'properties': {'osm_id': 413591726, 'osm_type': 2, 'distance': 473.17273413, 'category_ids': {'604': {'category_name': 'station', 'category_group': 'transport'}}, 'osm_tags': {'website': 'https://www.ptv.vic.gov.au/stop/1104/', 'name': 'Jolimont', 'wheelchair': 'yes'}}}]


In [ ]:
# VIC_COORDINATES = [-37.020100, 144.964600]
# m = folium.Map(location=VIC_COORDINATES, tiles='cartodbpositron', zoom_start=8)
# def fetch_isochrones(coordinate, retries=3, delay=10):
#     for attempt in range(retries):
#         try:
#             iso = client.isochrones(
#                 locations=[coordinate],
#                 profile='foot-walking',
#                 range=[900],
#                 validate=False
#             )
#             return iso['features']
#         except ApiError as e:
#             print(f"Error fetching isochrones for coordinate {coordinate}: {e}")
#             if attempt < retries - 1:
#                 time.sleep(delay)  # Wait before retrying
#                 delay *= 2  # Exponential backoff
#             else:
#                 print("Max retries reached. Skipping this coordinate.")
#                 return None
# def plot_isochrones(isochrones, map_obj):
#     if isochrones:
#         for isochrone in isochrones:
#             folium.Polygon(
#                 locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
#                 fill='00ff00',
#                 opacity=0.5
#             ).add_to(map_obj)
            
# def plot_map(coordinates, map_obj, output_file, request_limit_per_minute):
#     request_interval = 60 / request_limit_per_minute  # Time between requests to adhere to rate limit

#     for idx, coord in enumerate(coordinates):
#         if idx % 10 == 0 and idx > 0:  # Print progress message every 10 coordinates
#             print(f"Processed {idx} coordinates.")
#         isochrones = fetch_isochrones(coord)
#         plot_isochrones(isochrones, map_obj)
#         time.sleep(request_interval)  # Sleep to respect rate limiting

#     # Save the map to an HTML file
#     map_obj.save(output_file)
#     print(f"Map with isochrones saved to {output_file}")     
    
# output_file = '../plots/output_map.html'
# request_limit_per_minute = 10
# plot_map(coordinates, m, output_file, request_limit_per_minute)       

In [3]:
housing_file_1 = '../data/raw/coordinates_by_address_first_4500.json'
housing_file_2 = '../data/raw/coordinates_by_address_second_4500.json'
housing_file_3 = '../data/raw/coordinates_by_address_final_4500.json'


In [4]:
def filter_properties_with_coords(properties):
    filtered_properties = {}
    for key, value in properties.items():
        # Check if 'coordinates' exists and is not empty
        if 'coordinates' in value and value['coordinates']:
            filtered_properties[key] = value
    return filtered_properties

In [5]:
with open(housing_file_1) as f:
    coords_1 = json.load(f)

with open(housing_file_2) as f:
    coords_2 = json.load(f)
    
with open(housing_file_3) as f:
    coords_3 = json.load(f)
    
    
coords_1_filtered = filter_properties_with_coords(coords_1)
coords_2_filtered = filter_properties_with_coords(coords_2)
coords_3_filtered = filter_properties_with_coords(coords_3)


In [6]:
coordinates_dict = {}

for prop in [coords_1_filtered, coords_2_filtered, coords_3_filtered]:
    coordinates_dict.update(prop)

In [ ]:
with open('../data/raw/properties_with_coords.json', 'w') as f:
    json.dump(coordinates_dict, f, indent=4)

print("Filtered properties with coordinates saved to 'properties_with_coords.json'.")

In [8]:
gdf_regional = gpd.read_file("../data/PTV/", layer='PTV_REGIONAL_TRAIN_STATION')
gdf_regional = gdf_regional.to_crs(epsg=4326)

gdf_metro = gpd.read_file("../data/PTV/", layer='PTV_METRO_TRAIN_STATION')
gdf_metro = gdf_metro.to_crs(epsg=4326)


In [14]:
gdf_tram = gpd.read_file("../data/PTV_tram_bus/", layer='PTV_METRO_TRAM_STOP')
gdf_tram = gdf_tram.to_crs(epsg=4326)

gdf_bus = gpd.read_file("../data/PTV_tram_bus/", layer='PTV_METRO_BUS_STOP')
gdf_bus = gdf_bus.to_crs(epsg=4326)


In [15]:
def prepare_station_data(gdf):
    return gdf[['geometry', 'STOP_ID', 'STOP_NAME']].apply(
        lambda row: ((row['geometry'].y, row['geometry'].x), row['STOP_ID'], row['STOP_NAME']), axis=1
    ).tolist()

In [ ]:
train_station_gdf = gpd.GeoDataFrame(pd.concat([gdf_metro, gdf_regional], ignore_index=True))
tram_station_gdf = gpd.GeoDataFrame(gdf_tram)
bus_station_gdf = gpd.GeoDataFrame(gdf_bus)

train_station_data = prepare_station_data(train_station_gdf)
tram_station_data = prepare_station_data(tram_station_gdf)
bus_station_data = prepare_station_data(bus_station_gdf)


In [17]:
with open('../data/raw/properties_with_coords.json') as f:
   properties = json.load(f)

In [18]:
def find_closest_station(house_coords, stations):
    closest_station = None
    min_distance = float('inf')
    closest_station_id = None
    closest_station_name = None
    
    for (station_coords, station_id, station_name) in stations:
        distance = geodesic(house_coords, station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_station = station_coords
            closest_station_id = station_id
            closest_station_name = station_name
            
    return closest_station, closest_station_id, closest_station_name, min_distance

In [ ]:
# total_houses = len(properties)
# counter = 0
# # Add closest train station information to houses
# for house_url, house_info in list(properties.items()):
#     house_coords = (house_info['coordinates']['latitude'], house_info['coordinates']['longitude'])
#     closest_station_coords, closest_station_id, closest_station_name, distance = find_closest_station(house_coords, train_station_data)
    
#     if closest_station_coords:
#         properties[house_url]['closest_station'] = {
#             'coordinates': {'latitude': closest_station_coords[0], 'longitude': closest_station_coords[1]},
#             'stationID': closest_station_id,
#             'stationName': closest_station_name,
#             'distance_km': distance
#         }
#     else:
#         properties[house_url]['closest_station'] = {
#             'coordinates': {'latitude': None, 'longitude': None},
#             'stationID': None,
#             'stationName': None,
#             'distance_km': None
#         }
    
#     # Update progress every 1,000 houses
#     counter += 1
#     if counter % 1000 == 0:
#         print(f"Processed {counter} out of {total_houses} houses.")

In [ ]:
for idx, (house_url, house_info) in enumerate(properties.items(), start=1):
    house_coords = (house_info['coordinates']['latitude'], house_info['coordinates']['longitude'])
    
    # Find closest train station
    # closest_train_station_coords, closest_train_station_id, closest_train_station_name, train_distance = find_closest_station(house_coords, train_station_data)
    
    # Find closest tram station
    closest_tram_station_coords, closest_tram_station_id, closest_tram_station_name, tram_distance = find_closest_station(house_coords, tram_station_data)
    
    # Find closest bus station
    # closest_bus_station_coords, closest_bus_station_id, closest_bus_station_name, bus_distance = find_closest_station(house_coords, bus_station_data)
    
    # properties[house_url]['closest_train_station'] = {
    #     'coordinates': {'latitude': closest_train_station_coords[0], 'longitude': closest_train_station_coords[1]} if closest_train_station_coords else None,
    #     'stationID': closest_train_station_id,
    #     'stationName': closest_train_station_name,
    #     'distance_km': train_distance if closest_train_station_coords else None
    # }
    
    properties[house_url]['closest_tram_station'] = {
        'coordinates': {'latitude': closest_tram_station_coords[0], 'longitude': closest_tram_station_coords[1]} if closest_tram_station_coords else None,
        'stationID': closest_tram_station_id,
        'stationName': closest_tram_station_name,
        'distance_km': tram_distance if closest_tram_station_coords else None
    }
    
    # properties[house_url]['closest_bus_station'] = {
    #     'coordinates': {'latitude': closest_bus_station_coords[0], 'longitude': closest_bus_station_coords[1]} if closest_bus_station_coords else None,
    #     'stationID': closest_bus_station_id,
    #     'stationName': closest_bus_station_name,
    #     'distance_km': bus_distance if closest_bus_station_coords else None
    # }
    
    # Print progress every 1000 houses
    if idx % 1000 == 0:
        print(f"Processed {idx} houses...")


In [ ]:
properties

In [ ]:
with open('../data/raw/houses_with_stations.json', 'w') as file:
    json.dump(properties, file, indent=4)

print("Finished processing all houses.")

In [ ]:
gdf_tram